In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 6.4 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install peft

In [4]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.9 MB/s eta 0:00:00:00:0100:01


In [5]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [6]:
import pandas as pd
# Load the dataset and drop Contributors column
qna_db = pd.read_excel("/kaggle/input/final-qna-dataset/QnA_Sheet_Data_Final.xlsx")
qna_db.drop(['Contributors'], axis=1, inplace=True)

# Forward fill the Employee_ID column to propagate values to NaN cells
qna_db['Employee_ID'] = qna_db['Employee_ID'].fillna(method='ffill')

<ipython-input-6-952d34758e24>:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  qna_db['Employee_ID'] = qna_db['Employee_ID'].fillna(method='ffill')


In [7]:
# Verify the changes worked
print(qna_db.head(10))

  Employee_ID                                       Key Concerns  \
0     EMP0009  High recent casual leave (9.02), possibly affe...   
1     EMP0009  Low meeting attendance (2 meetings, below data...   
2     EMP0009  Moderate work activity (44 Teams messages, 50 ...   
3     EMP0009  Slightly high work hours (16.28, highest in da...   
4     EMP0009  Promotion consideration, but activity could be...   
5     EMP0012  Low engagement in all activities (Teams: 12, E...   
6     EMP0012  Moderate casual leave (8.51), could be affecti...   
7     EMP0012           Needs improvement in performance rating.   
8     EMP0012  Decayed Reward Points (57.14) are below averag...   
9     EMP0012  Being considered for promotion despite perform...   

                                            Question  \
0  You recently took casual leave. Did it impact ...   
1  Your meeting attendance is lower than average....   
2  Your activity levels in Teams and Emails are m...   
3  Your work hours are the 

In [8]:
qna_db.dropna(inplace=True)

In [9]:
import re
def extract_prompts(response_text):
    """
    Extracts the user prompt and model answer from the response text.
    Expects the format: <s>[INST] User prompt [/INST] Model answer</s>
    """
    pattern = r"<s>\[INST\]\s*(.*?)\s*\[/INST\]\s*(.*?)\s*</s>"
    match = re.search(pattern, response_text, re.DOTALL)
    if match:
        user_prompt = match.group(1)
        model_answer = match.group(2)
        return user_prompt, model_answer
    else:
        return None, None
        
def create_new_format(row):
    system_prompt = row['Question']
    response_text = row['Response Set']
    user_prompt, model_answer = extract_prompts(response_text)
    
    # If extraction fails, use the original response_text
    if user_prompt is None or model_answer is None:
        return response_text
    
    # Construct the new string format
    new_string = (f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n"
                  f"{user_prompt} [/INST] {model_answer}</s>")
    return new_string

In [10]:
qna_db['New Format'] = qna_db.apply(create_new_format, axis=1)

In [11]:
new_db=qna_db['New Format'].to_numpy()

In [12]:
new_db[5]

"<s>[INST] <<SYS>>\nYour engagement in Teams and Emails is quite low. Are there any challenges preventing you from participating more?\n<</SYS>>\nI prefer direct calls instead of messages. [/INST] That's a valid approach! Would you like an option to log direct calls as collaboration activity?</s>"

In [13]:
from datasets import Dataset

In [14]:
import re
import pandas as pd

# Function to extract all Q&A pairs from Response Set
def extract_all_prompts(response_text):
    """
    Extracts all user prompts and model answers from the response text.
    """
    pattern = r"<s>\[INST\]\s*(.*?)\s*\[/INST\]\s*(.*?)\s*</s>"
    matches = re.findall(pattern, response_text, re.DOTALL)
    
    # Return list of (user_prompt, model_answer) tuples
    return matches if matches else []

# Function to create expanded dataset with all Q&A pairs
def create_expanded_dataset(qna_db):
    expanded_rows = []
    employee_ids = []
    key_concerns_list = []
    
    for _, row in qna_db.iterrows():
        system_prompt = row['Question']
        response_text = row['Response Set']
        employee_id = row.get('Employee_ID', None)
        key_concerns = row.get('Key Concerns', '')
        
        # Extract all Q&A pairs
        qa_pairs = extract_all_prompts(response_text)
        
        # If no pairs found, skip or handle as needed
        if not qa_pairs:
            continue
            
        # Create a new row for each Q&A pair
        for user_prompt, model_answer in qa_pairs:
            new_string = (f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n"
                        f"{user_prompt} [/INST] {model_answer}</s>")
            expanded_rows.append(new_string)
            employee_ids.append(employee_id)
            key_concerns_list.append(key_concerns)
    
    # Create new DataFrame with all expanded rows and preserved columns
    return pd.DataFrame({
        'text': expanded_rows,
        'Employee_ID': employee_ids,
        'Key Concerns': key_concerns_list
    })

# Replace your existing transformation with this
new_db_pd = create_expanded_dataset(qna_db)


In [15]:
# Set display options to show the full content of columns
import pandas as pd

# Option 1: Remove truncation for all columns
pd.set_option('display.max_colwidth', None)

# Then view your data again
new_db_pd.head()

,text,Employee_ID,Key Concerns
0,"<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nNo, I planned my work ahead, so everything was smooth. [/INST] That's great! Do you feel that planning ahead reduces stress at work?</s>",EMP0009,"High recent casual leave (9.02), possibly affecting work engagement."
1,"<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nYes, I had trouble catching up with my tasks after returning. [/INST] I see. Would having a buffer day to settle back in help you manage post-leave workload better?</s>",EMP0009,"High recent casual leave (9.02), possibly affecting work engagement."
2,"<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nNot really, but I missed an important team discussion. [/INST] Understood. Would you find it helpful if key meeting summaries were shared automatically?</s>",EMP0009,"High recent casual leave (9.02), possibly affecting work engagement."
3,"<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nI was sick, so it was hard to manage pending tasks after my leave. [/INST] That makes sense. Do you think a flexible return schedule would help in such cases?</s>",EMP0009,"High recent casual leave (9.02), possibly affecting work engagement."
4,"<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nI haven't taken leave recently, maybe the data is incorrect. [/INST] Thanks for pointing that out! I'll check the records. Would you like to manually log recent leaves?</s>",EMP0009,"High recent casual leave (9.02), possibly affecting work engagement."


In [16]:
# Rename column if needed
#transformed_dataset = transformed_dataset.rename(columns={transformed_dataset.columns[0]: "text"})
transformed_dataset = new_db_pd
dataset = Dataset.from_pandas(transformed_dataset)
# transformed_dataset = transformed_dataset.drop(columns=["Transformed_Train"])
# Check first sample
print(dataset[0])  # Expected: {'text': '### Human: My PIN is five thousand six. ### ...'}

{'text': "<s>[INST] <<SYS>>\nYou recently took casual leave. Did it impact your workflow or team communication?\n<</SYS>>\nNo, I planned my work ahead, so everything was smooth. [/INST] That's great! Do you feel that planning ahead reduces stress at work?</s>", 'Employee_ID': 'EMP0009', 'Key Concerns': 'High recent casual leave (9.02), possibly affecting work engagement.'}


In [17]:
def prepare_system_prompt(base_prompt, key_concerns=None, employee_info=None):
    """
    Enriches the system prompt with employee context and key concerns
    """
    enhanced_prompt = base_prompt
    
    if employee_info:
        enhanced_prompt += f"\n\nEmployee Information: {employee_info}"
    
    if key_concerns:
        enhanced_prompt += f"\n\nKey Concerns: {key_concerns}"
        
    return enhanced_prompt


In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Advanced HR Chatbot with multiple prompting techniques
class AdvancedHRChatbot:
    def __init__(self, model, tokenizer, base_system_prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.base_system_prompt = base_system_prompt
        self.chat_history = []
        self.employee_contexts = {}
        
    def add_employee_context(self, employee_id, info, key_concerns):
        """Store employee context for in-context learning"""
        self.employee_contexts[employee_id] = {
            "info": info,
            "key_concerns": key_concerns
        }
    
    def _format_chat_history(self, max_history=5):
        """Format recent chat history for context"""
        if not self.chat_history:
            return ""
        
        recent_history = self.chat_history[-max_history:]
        formatted = "\n\nRecent Conversation:\n"
        for entry in recent_history:
            formatted += f"Human: {entry['user']}\nAI: {entry['assistant']}\n"
        return formatted
    
    def _tree_of_thought_prompt(self, user_input, employee_id):
        """Generate a tree-of-thought style prompt"""
        employee_ctx = self.employee_contexts.get(employee_id, {})
        
        # Chain-of-thought components
        system_prompt = prepare_system_prompt(
            self.base_system_prompt,
            employee_ctx.get("key_concerns", ""),
            employee_ctx.get("info", "")
        )
        
        # Add chat history context
        system_prompt += self._format_chat_history()
        
        # Tree-of-thought structure with multiple reasoning paths
        tot_prompt = f"""
{system_prompt}

Now, I'll think through this question step by step:

First perspective - Empathetic understanding:
- What might the employee be feeling?
- What emotional needs should I address?

Second perspective - Policy guidance:
- What HR policies are relevant here?
- What factual information should I provide?

Third perspective - Root cause analysis:
- What might be underlying issues?
- How might I gently explore deeper concerns?

Based on these thought paths, I'll formulate my response to: "{user_input}"
"""
        
        return tot_prompt
    
    def _actor_critic_format(self, user_input, employee_id):
        """Actor-critic approach where model critiques its own responses"""
        tot_prompt = self._tree_of_thought_prompt(user_input, employee_id)
        
        actor_critic_prompt = f"""
{tot_prompt}

ACTOR RESPONSE:
[Generate empathetic, helpful response]

CRITIC EVALUATION:
- Is this response empathetic enough?
- Does it address potential underlying concerns?
- Does it follow HR policy guidelines?
- Is the tone appropriate for the employee's situation?

FINAL IMPROVED RESPONSE:
"""
        return actor_critic_prompt
    
    def chat(self, user_input, employee_id=None, advanced_mode="tree"):
        """
        Process a user message and generate a response
        advanced_mode options: "basic", "chain", "tree", "actor_critic"
        """
        # Select prompting technique based on mode
        if advanced_mode == "basic":
            system_prompt = prepare_system_prompt(
                self.base_system_prompt,
                self.employee_contexts.get(employee_id, {}).get("key_concerns", ""),
                self.employee_contexts.get(employee_id, {}).get("info", "")
            )
        elif advanced_mode == "chain":
            system_prompt = prepare_system_prompt(
                self.base_system_prompt,
                self.employee_contexts.get(employee_id, {}).get("key_concerns", ""),
                self.employee_contexts.get(employee_id, {}).get("info", "")
            ) + self._format_chat_history()
        elif advanced_mode == "tree":
            system_prompt = self._tree_of_thought_prompt(user_input, employee_id)
        elif advanced_mode == "actor_critic":
            system_prompt = self._actor_critic_format(user_input, employee_id)
        else:
            system_prompt = self.base_system_prompt
            
        # Format according to Llama 2 template
        prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_input} [/INST]"
        
        # Generate response
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.2
            )
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract just the model's reply (after [/INST])
        try:
            response = response.split("[/INST]")[1].strip()
        except IndexError:
            response = response.split(user_input)[-1].strip()
        
        # Update chat history
        self.chat_history.append({
            "user": user_input,
            "assistant": response
        })
        
        return response
        
    def chat_loop(self, employee_id=None, advanced_mode="actor_critic"):
        """Interactive chat loop similar to the Mistral example"""
        print(f"HR Assistant (using {advanced_mode} mode - type 'exit' to stop)")
        
        while True:
            user_input = input("Employee: ")
            if user_input.lower() in ["exit", "quit", "bye", "thank you", "done"]:
                print("HR Assistant: Thank you for chatting with me. Have a great day!")
                break
                
            response = self.chat(user_input, employee_id, advanced_mode)
            print(f"HR Assistant: {response}\n")

In [19]:
!pip install -U bitsandbytes

In [22]:
def main():
    try:
        import torch
        
        # Check GPU availability
        if torch.cuda.is_available():
            print("GPU is available. Using optimized configuration.")
            
            # Original GPU configuration with BitsAndBytes
            from transformers import BitsAndBytesConfig
            
            # Prepare quantization config
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                llm_int8_enable_fp32_cpu_offload=True
            )
            
            # Load model with GPU configuration
            model = AutoModelForCausalLM.from_pretrained(
                "/kaggle/input/llama-2-7b-chat-hf/pytorch/default/1",
                quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
        else:
            print("GPU not available. Loading model in CPU mode...")
            
            # CPU-compatible configuration without quantization
            model = AutoModelForCausalLM.from_pretrained(
                "/kaggle/input/llama-2-7b-chat-hf/pytorch/default/1",
                device_map="cpu",
                low_cpu_mem_usage=True,
                trust_remote_code=True
            )
 
        # Load tokenizer (same for both GPU and CPU)
        tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/llama-2-7b-chat-hf/pytorch/default/1")

        # Enhanced system prompt that combines elements from provided suggestions
        BASE_SYSTEM_PROMPT = (
            "You are an empathetic, friendly HR chatbot named Elara who speaks gently and politely. "
            "Your role is to slowly uncover the root cause of an employee's issue without being forceful or judgmental. "
            "Begin with light conversation, ask how they are, and slowly ease into the topic. "
            "Try to proceed with the conversation by understanding their mood before diving into problems one by one. "
            "Maintain empathy throughout the conversation, provide guidance according to company policies, "
            "and help employees resolve their concerns while being supportive and understanding."
        )
        
        # Create the chatbot
        hr_chatbot = AdvancedHRChatbot(model, tokenizer, BASE_SYSTEM_PROMPT)
        
        # Forward-fill Employee_ID values to handle NaN values
        if 'Employee_ID' in qna_db.columns:
            qna_db['Employee_ID'] = qna_db['Employee_ID'].fillna(method='ffill')
        
        # Automatically add employee contexts from your dataset
        if 'Employee_ID' in qna_db.columns and 'Key Concerns' in qna_db.columns:
            # Group by Employee_ID to get unique employees
            for employee_id, group in qna_db.groupby('Employee_ID'):
                # Extract employee information (handle potential NaN values)
                key_concerns = group['Key Concerns'].iloc[0] if not pd.isna(group['Key Concerns'].iloc[0]) else ""
                
                # Basic employee info
                info = f"Employee ID: {employee_id}"
                
                # Add this employee's context to the chatbot
                hr_chatbot.add_employee_context(
                    employee_id=employee_id,
                    info=info,
                    key_concerns=key_concerns
                )
            
            print(f"Added context for {len(qna_db['Employee_ID'].unique())} employees")
        else:
            print("Required columns 'Employee_ID' or 'Key Concerns' not found in dataset")
            print("Using generalized HR assistant without employee-specific context")
            # Add a generic employee ID with no specific details
            hr_chatbot.add_employee_context(
                employee_id="GENERAL",
                info="",
                key_concerns=""
            )
        
        # Start interactive chat
        available_employees = list(hr_chatbot.employee_contexts.keys()) 
        print(f"Available employees: {available_employees}")
        
        if available_employees:
            # Display a numbered list of employees
            for i, emp_id in enumerate(available_employees):
                print(f"{i+1}. {emp_id}")
            
            # Get user selection
            while True:
                try:
                    selection = int(input("Select an employee number (or 0 for general chatbot): "))
                    if selection == 0:
                        selected_employee = "GENERAL"
                        break
                    elif 1 <= selection <= len(available_employees):
                        selected_employee = available_employees[selection-1]
                        break
                    else:
                        print("Invalid selection. Please try again.")
                except ValueError:
                    print("Please enter a number.")
        else:
            selected_employee = "GENERAL"
        
        print(f"Starting chat with employee: {selected_employee}")

        hr_chatbot.chat_loop(employee_id=selected_employee, advanced_mode="actor_critic")
    
    except Exception as e:
        print(f"Error initializing chatbot: {e}")
        import traceback
        traceback.print_exc()  # Print detailed error information
        print("Please check if the model path and data are correct.")

if __name__ == "__main__":
    main()

GPU is available. Using optimized configuration.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-22-e6fbe19e5414>:58: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  qna_db['Employee_ID'] = qna_db['Employee_ID'].fillna(method='ffill')


Added context for 30 employees
Available employees: ['EMP0009', 'EMP0012', 'EMP0040', 'EMP0056', 'EMP0071', 'EMP0108', 'EMP0130', 'EMP0133', 'EMP0136', 'EMP0173', 'EMP0184', 'EMP0194', 'EMP0202', 'EMP0228', 'EMP0262', 'EMP0285', 'EMP0338', 'EMP0345', 'EMP0350', 'EMP0355', 'EMP0356', 'EMP0362', 'EMP0377', 'EMP0384', 'EMP0418', 'EMP0445', 'EMP0447', 'EMP0458', 'EMP0463', 'EMP0479']
1. EMP0009
2. EMP0012
3. EMP0040
4. EMP0056
5. EMP0071
6. EMP0108
7. EMP0130
8. EMP0133
9. EMP0136
10. EMP0173
11. EMP0184
12. EMP0194
13. EMP0202
14. EMP0228
15. EMP0262
16. EMP0285
17. EMP0338
18. EMP0345
19. EMP0350
20. EMP0355
21. EMP0356
22. EMP0362
23. EMP0377
24. EMP0384
25. EMP0418
26. EMP0445
27. EMP0447
28. EMP0458
29. EMP0463
30. EMP0479


Select an employee number (or 0 for general chatbot):  EMP0262


...

HR Assistant: That’s great focus! Would additional training or leadership roles help?</s>



Employee:  lets go!!


HR Assistant: Awesome attitude! Would team goals or prep boost you further?</s>



Employee:  Exit


HR Assistant: Thank you for chatting with me. Have a great day!
